## Fitting an LSTM to Stock Price Data

In [1]:
#import necessary packages
'''You may need to download pytorch and plotly if you don't already have them '''

import numpy as np 
import pandas as pd
from sklearn import *
import sys

import torch
from torch import nn
import torch.utils.data as data_utils

import datetime
from time import sleep
from threading import Timer
import time

from IPython.display import clear_output
import plotly.graph_objects as go
import plotly


import json
import urllib.request

from threading import Thread
from multiprocessing import Process


In [2]:
#read in data as pandas dataframe, then convert to numpy array
msft_16 = pd.read_csv(r'C:\Users\Megan Finley\Documents\Spring2020\M567_Projects\Data\Microsoft\msft_4_16_clean.txt',
                      header=0)

#extract price column from data frame, it will be whatever name you named the price column
msft_16_price = msft_16['salePrice']

#extract time column from data frame, it will be whatever name you named the time column
msft_16_time = msft_16['time']


#convert to numpy array
msft_16_price = msft_16_price.to_numpy()
msft_16_time = msft_16_time.to_numpy()



## Functions

In [23]:
#below creates functions to grab the training and test data based on above parameters
def training_data(data,train_window,starting_timestep):    
    str_price = data[starting_timestep:train_window+starting_timestep]
    price = [float(x) for x in str_price]
    diff_1 = [j-i for i, j in zip(price[:-1], price[1:])] #to produce the differences 1 observation a part
    diff_2 = [price[i+2]-price[i] for i in range(len(price)-2)] #to produce the differences 2 observations a part
    
    #create matrix of features   
    x_train = np.c_[diff_1[1:-1],diff_2[:-1]] 
   
    str_y_train = data[starting_timestep:train_window+starting_timestep]
    y_train = [float(x) for x in str_y_train]
    y_train = y_train[3:] #grab prices from index 3 up since we lose our first 3 observations from taking differences
    
    #cast data to tensors, an object pytorch accepts
    x_train = torch.from_numpy(x_train)
    y_train = torch.tensor(y_train)
    x_train = x_train.to(torch.float32)
    y_train = y_train.to(torch.float32)

    return x_train, y_train

def test_data(data,train_window,time_ahead,starting_timestep):
    str_price = data[train_window+starting_timestep-3:train_window+starting_timestep+time_ahead]
    price = [float(x) for x in str_price]
    diff_1 = [j-i for i, j in zip(price[:-1], price[1:])] #to produce the differences 1 observation a part
    diff_2 = [price[i+2]-price[i] for i in range(len(price)-2)] #to produce the differences 2 observations a part
    
    #create matrix of features 
    x_test = np.c_[diff_1[1:-1],diff_2[:-1]] 
   
    str_y_test = data[train_window+starting_timestep-3:train_window+starting_timestep+time_ahead]
    y_test = [float(x) for x in str_y_test]

    y_test = y_test[3:] #grab prices from index 3 up since we lose our first 3 observations from taking differences
    
    #cast data to tensors, an object pytorch accepts
    x_test = torch.from_numpy(x_test)
    y_test = torch.tensor(y_test)
    x_test = x_test.to(torch.float32)
    y_test = y_test.to(torch.float32)
    return x_test, y_test

def get_data(token,tag,file_name,price_list,time_list):    
    try:
        #request to pull info from iex 
        html = urllib.request.urlopen("https://cloud.iexapis.com/stable/tops?token="+token+"&symbols="+tag)
        data = json.loads(html.read().decode('utf-8'))

        #extract data from dictionary within list and convert to string so we can write text file
        salePrice = str(data[0]['lastSalePrice'])
        print(salePrice)

        now = datetime.datetime.now() #extract the timestamp from the computer
        hour = '{:02d}'.format(now.hour) #extract hour fom datetime call
        minute = '{:02d}'.format(now.minute)
        time = hour+':'+minute #time
        
        price_list.append(salePrice) #append price to list
        time_list.append(time) #apend time to list
        
        #write data to file so we have a record of it
        with open(file_name,'a') as f:
            f.write(time+','+salePrice+'\n')
    except:
    
        pass
    
    return price_list, time_list

#prediction function
def predict(x_test, fut_pred):
    for i in range(fut_pred):
        seq = x_test
        with torch.no_grad():
            model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                            torch.zeros(1, 1, model.hidden_layer_size))
            prediction = model(seq).item()
            
    return prediction

#function to write predictions to file
def write(times,actual_prices,predicted_prices,file_name):
   
    with open(file_name, 'w') as f:
        f.write('time'+','+'actualPrice'+','+'predictedPrice'+'\n')
        for i in range(len(times)):
            f.write(str(times[i])+','+str(actual_prices[i])+','+str(predicted_prices[i])+'\n')

#function to update time for real time analysis
def update_time():
    now = datetime.datetime.now() #extract the timestamp from the computer
    hour = '{:02d}'.format(now.hour) #extract hour from datetime call
    minute = '{:02d}'.format(now.minute) #extract minute from datetime call
    time = hour+':'+minute #concatenate hour and minute 
    return time

#function to get the time for our prediction
def prediction_time():
    now = datetime.datetime.now() #extract timestamp from computer
    hour = '{:02d}'.format(now.hour) #extract hour from datetime call
    minute = '{:02d}'.format(now.minute) #extract minute from datetime call
    #logic to check the minute, cannot add one to the 59th minute
    if int(minute) < 9:
        next_minute = int(minute) + 1
        time = hour +':'+ '0'+ str(next_minute)
    elif minute == '59':
        next_hour = int(hour)+1
        next_minute = '00'
        time = str(next_hour)+':'+str(next_minute)  
    else:
        next_minute = int(minute) + 1
        time = hour +':'+ str(next_minute)
    
    return time

#below is a function that will allow the user to make their own decisions when to buy sell or hold
def carryout_decision(current_price,share_bank,wallet,n,prediction,initial_investment,continue_trading,hold_end,long_hold):
   
    global bought_price #will keep track of the last time we bought so we don't sell and lose money
    global last_action #keeps track of the last action we took for recommendation purposes
    global profits #keeps track of our profits
    
    #below logic sets up the initial recommendation and bought price which doesnt exist on the first iteration
    if n == 0:
        bought_price = None
        last_action = None
        profits = 0
    
    #checks to see if we have a long hold in place    
    if n < hold_end:
        long_hold = True
    else: 
        long_hold = False
        hold_end = 0 #reset hold_end to zero so we can proceed with making decisions
    
    #initialize proceed to true so decisions can be made
    proceed = True
    
    if continue_trading == True:
        if long_hold == False:
            while proceed == True:            
                #prompt user with decision and give some helpful information to orient the decision
                print(f'Please decide whether to buy, sell, hold, or terminate. Shares were',bought_price,
                      'the last time you bought shares.')
                decision = input() #prompts decision
                #logic needed to ensure the user has inputed a valid response
                if decision == 'buy' or decision == 'sell' or decision == 'hold': 
                    #logic to handle the process of buying shares and making sure input is valid
                    if decision == 'buy':  
                        valid = False
                        while valid == False:
                            print('Please input how many shares you would like to buy.')
                            shares_buy = input()
                            #check if input is valid
                            try:
                                shares_buy = int(shares_buy)
                                valid = True
                            except ValueError:
                                print('Not a valid input. Input a number')
                        #logic to ensure funds and choices are valid, will update how many shares and the funds availabe
                        if shares_buy*current_price <= wallet:
                            wallet -= shares_buy*current_price
                            share_bank += shares_buy
                            bought_price = current_price
                            proceed = False #terminates the function and lets forecasting proceed
                            last_action = 'buy'
                            print('You just purchased', shares_buy, 'shares for', bought_price,
                                  'a share. You now have', share_bank, 'shares and', "${:.2f}".format(wallet), 'left.')
                        elif shares_buy*current_price > wallet:
                            afford = int(wallet/current_price)
                            valid = False
                            print('Error: Insufficient funds - You can only afford', afford, 'shares.')
                        else:
                            print('Input not understood')
                    #logic to carry out the selling action works similarly to buying 
                    if decision == 'sell':
                        valid = False
                        while valid == False:
                            print('Please input how many shares you would like to sell.')
                            shares_sell = input()
                            #check if input is valid
                            try:
                                shares_sell = int(shares_sell)
                                valid = True
                            except ValueError:
                                print('Not a valid input. Input a number')   
                        if share_bank != 0: #make sure we own shares
                            if shares_sell <= int(share_bank) and shares_sell > 0:
                                wallet += current_price*shares_sell #carryout sale
                                share_bank -= shares_sell #reduce number of shares in bank
                                transaction_profit = wallet - initial_investment #calculate transaction profit
                                if wallet > initial_investment: #if profits are not negative carry out the following
                                    profits += transaction_profit #add to our profits
                                    wallet = initial_investment #reset wallet to initial investment
                                current_profits = (wallet-initial_investment) + profits #otherwise calculate profits
                                proceed = False #end decision making process
                                last_action = 'sell'
                                print('You now have ',share_bank,'shares and',"${:.2f}".format(wallet),
                                      'left. You made a profit of',"${:.2f}".format(transaction_profit), 
                                      'for a total of',"${:.2f}".format(current_profits), 'in profits.')
                            elif shares_sell > share_bank:
                                print('Error: You cannot sell more shares than you own. Reenter number of shares to sell.')
                            elif shares_sell == 0:
                                print('You did not sell any shares.')
                                proceed = False
                            else:
                                print('Input not understood')
                        else:
                            print('Error: You do not own any shares in this stock')    
                    #logic to hold and for how long
                    if decision == 'hold':
                        valid = False
                        #make sure input is valid
                        while valid == False: 
                            print('How long would you like to hold for. Please enter in minutes.')
                            hold_time = input()
                            try:
                                hold_time = int(hold_time) #how long the user wishes to hold/not make decision
                                hold_end = n + hold_time #use current iteration to set when the hold ends
                                valid = True
                            except ValueError:
                                print('Not a valid input. Input a number')
                                   
                            print('You have',share_bank,'shares and',"${:.2f}".format(wallet),'.')
                            long_hold = True #set long hold boolean so we are not prompted to make another decision
                            proceed = False #end decision making process
                            last_action = 'hold'             

                elif decision == 'terminate':
                    #check to see if user owns shares, then execute sale logic
                    if share_bank != 0:
                        wallet += current_price*share_bank #carryout sale
                        share_bank = 0 #set share bank to 0
                        transaction_profit = wallet - initial_investment #calculate transaction profit
                        profits += transaction_profit #add to running total of the profit                  
                   
                    #will interrupt the while loop so the user can stop trading
                    #forecasting and plotting will continue
                    continue_trading = False 
                    proceed = False #end decision making process
                    print("You have finished trading for the day. You made",profits)
                else:
                    #error message if user does not input a valid option
                    print('Input not understood')            

    return wallet, share_bank, profits,continue_trading, hold_end, long_hold
    
        

## What is an LSTM?

In short, an LSTM (Long Short-Term Memory) is a type of RNN that will remember and forget as the model trains. The benefit of an LSTM is that it learns long-term behavior easily. An LSTM takes the previous hidden state, previous cell state and current data features as inputs. Below we create an LSTM where the outputs of the LSTM layer will be a hidden state and cell state at current time step t along with an output. The output gets passed to the linear layer where it is then stored as a prediction.

The cell state is important because it carries the information we decide to remember and forget. It comes from the current informtion and previous hidden state. 

There is a great [blog post by Christopher Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) about LSTM's and RNN's.

There is also a [very helpful time series](https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/) LSTM tutorial by Usman Malik from Stack Abuse.


In [4]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size #how many hidden layer nodes we have
        self.lstm = nn.LSTM(input_size, hidden_layer_size) #initialize an LSTM model
        
        #linear is like the identity when predicting numeric values
        self.linear = nn.Linear(hidden_layer_size, output_size) 

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size), #initialize hidden cells
                            torch.zeros(1,1,self.hidden_layer_size))

    #passing the data through the RNN
    def forward(self, input_seq):
        #the forward pass will give us an out value and a value from the hidden cell
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell) 
        
        #prediction from the lstm
        prediction = self.linear(lstm_out.view(len(input_seq), -1))
        

        return prediction

## Run Model

**Note: can comment out lines 118 through 120 to allow model to just forecast and plot**

In [27]:
#initialize the model
model = Model(input_size=2, hidden_layer_size=32, output_size=1)
#instantiate the loss function
loss_function = nn.MSELoss()
#choose the gradient descent optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

epochs = 500 #number of epochs we will train for
fut_pred = 1 #number of time steps predicting ahead

token ='pk_1cac00fb277a4ef59c1101965078674b' #token needed to gain access to IEX
tag = 'msft' #stock that IEX will get data for
file_name = 'msft_data_5_6.txt' #text file name to write data to

#initialize lists for data
predictions = [] 
times = []
actual_prices = []
prediction_times = []
msft_prices = []
msft_times = []

#initialize variables to pass into carryout_decision function
initial_investment = 1000
money = initial_investment
available_shares = 0
days_profits = 0
continue_trading = True
hold_end = 0
long_hold = False


lost_obvs = 3 #how many observations we will lose if we use first and second order differences
train_window = 10 + lost_obvs #how many observations we want to use to train the model
time_ahead = 1  #how many predictions/minutes ahead we want to predict
starting_timestep = 0 #where we want to start running the model
n = 0 #keep track of iterations

stop_time = '14:00' #specify the time the alogorithm will stop
time = update_time() #initialize time

#get data, train model, plot data, carry out decisions
while time != stop_time:
    time = update_time() #need to constantly update the time
    #logic to check to see if we already have an observation for the current time
    if time not in msft_times:
        #make the call to get the data
        msft_prices, msft_times = get_data(token,tag,file_name,msft_prices,msft_times)
       
        #can start training the model and making predictions once there is enough data
        if len(msft_prices) > 14:
            #training data
            x_train, y_train = training_data(msft_prices, train_window, starting_timestep)
            
            #testing data
            x_test, y_test = test_data(msft_prices, train_window, time_ahead, starting_timestep)   
            
            #puts training and test data into a format acccepted by pytorch
            train = data_utils.TensorDataset(x_train, y_train)
            train_loader = data_utils.DataLoader(train, batch_size=10, shuffle=False)

            test = data_utils.TensorDataset(x_test, y_test)
            test_loader = data_utils.DataLoader(test, batch_size=1, shuffle=False)   
            
            #put the current time in a list for plotting purposes
            times.append(msft_times[starting_timestep+ train_window+1])
            
            #get the actual price
            actual_prices.append(float(msft_prices[starting_timestep+ train_window+1]))
            
            #get the time 1 minute in the future and append to list for plotting
            future_time = prediction_time()
            prediction_times.append(future_time)
           
            #iterate over the number of epochs
            for i in range(epochs):
                #set hidden cell state to zeros
                model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                                     torch.zeros(1, 1, model.hidden_layer_size))
                
                #iterate over each data point in the training sequence
                for data, price in train_loader:
                    #zero out the gradient
                    optimizer.zero_grad() 
                    
                    #make a prediction
                    y_pred = model(data)  
                   
                    #need to use squeeze to fix dimensionality warning, the tensor is [input_len,1]
                    #squeeze drops the 1
                    y_pred = torch.squeeze(y_pred,-1) 
                   
                    #calculate loss
                    loss = loss_function(y_pred, price)
                    
                    #propagate loss backward through model
                    loss.backward(retain_graph = True)
                    
                    #update our weights that we are training in the model
                    optimizer.step()

            #make predictions after the model has trained for the particular window
            prediction = predict(x_test, fut_pred)
            predictions.append(float(prediction))
            time = update_time() #update time

            #plot the prices and predictions and update the figure every iteration
            clear_output(wait=True)
            fig = go.Figure()
            fig.add_trace(go.Scatter(x= times, y= actual_prices, mode= 'lines+markers', name= 'Actual Price'))
            fig.add_trace(go.Scatter(x = prediction_times, y = predictions, mode = 'lines', name = 'Predictions'))
            fig.update_layout(title = 'Actual vs Forecasted Prices for Microsoft')
            fig.show()

            #carry out own decisions of whether or not to buy sell or hold
            print('The predicted price for',prediction_times[n], 'is',"{:.2f}".format(prediction),
                  'and the current price is',actual_prices[n])
            money, available_shares,days_profits,continue_trading,hold_end,long_hold = carryout_decision(actual_prices[n],
                                                                    available_shares,money,n,predictions[n],
                                                                    initial_investment,continue_trading,hold_end,long_hold)

            #increment timestep so train window can be advanced
            starting_timestep +=1
            n += 1    

## Run Model on Static File

**Can comment out lines 83 through 86 to allow model to just forecast and plot**

In [24]:
#initialize the model
model = Model(input_size = 2, hidden_layer_size = 32, output_size = 1)

#instantiate the loss function
loss_function = nn.MSELoss()

#choose the gradient descent optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1)

epochs = 500 #number of epochs we will train for
fut_pred = 1 #number of time steps predicting ahead

#initialize lists for data
predictions = []
times = []
actual_prices = []
prediction_times = []
msft_prices = []
msft_times = []

#initialize variables to pass into carryout_decision function
initial_investment = 1000
money = initial_investment
available_shares = 0
days_profits = 0
continue_trading = True
hold_end = 0
long_hold = False

lost_obvs = 3 #how many observations we will lose if we use first and second order differences
train_window = 10 + lost_obvs #how many observations we want to use to train the model
time_ahead = 1  #how many predictions/minutes ahead we want to predict
starting_timestep = 0 #where we want to start running the model
n = 0 #keep track of iterations


while train_window+starting_timestep+time_ahead+1 != len(msft_16_price):
    x_train, y_train = training_data(msft_16_price, train_window, starting_timestep)
    x_test, y_test = test_data(msft_16_price, train_window, time_ahead, starting_timestep)   

    train = data_utils.TensorDataset(x_train, y_train)
    train_loader = data_utils.DataLoader(train, batch_size=10, shuffle=False)

    test = data_utils.TensorDataset(x_test, y_test)
    test_loader = data_utils.DataLoader(test, batch_size=1, shuffle=False)   
    times.append(msft_16_time[starting_timestep+ train_window + time_ahead])
    actual_prices.append(msft_16_price[starting_timestep+ train_window + time_ahead])
    prediction_times.append(msft_16_time[starting_timestep+ train_window + time_ahead+1])
    
    #iterate over the number of epochs
    for i in range(epochs):
        #iterate over each data point in the training sequence
        #set hidden cell state to zeros
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),torch.zeros(1, 1, model.hidden_layer_size))
        for data, price in train_loader:
            #zero out the gradient
            optimizer.zero_grad() 
            #made a prediction
            y_pred = model(data)  
            #need to use squeeze to fix dimensionality warning, the tensor is [input_len,1], squeeze drops the 1
            y_pred = torch.squeeze(y_pred,-1) 
            #calculate loss
            loss = loss_function(y_pred, price)
            #propagate loss backward through model
            loss.backward(retain_graph = True)
            #update our weights that we are training in the model
            optimizer.step()

    #make predictions after the model has trained for the particular window
    prediction = predict(x_test, fut_pred)
    predictions.append(float(prediction))

    #plot the prices and predictions and update the figure every iteration
    clear_output(wait=True)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x= times, y= actual_prices, mode= 'markers', name= 'Actual Price'))
    fig.add_trace(go.Scatter(x = prediction_times, y = predictions, mode = 'lines', name = 'Predictions'))
    fig.update_layout(title = 'Actual vs Predicted Prices for Facebook 3/27/2020')
    fig.show()
    
    print('The predicted price for',prediction_times[n], 'is',"{:.2f}".format(prediction),
          'and the current price is',actual_prices[n])
    money, available_shares,days_profits,continue_trading,hold_end,long_hold = carryout_decision(actual_prices[n],
                                                                    available_shares,money,n,predictions[n],
                                                                    initial_investment,continue_trading,hold_end,
                                                                                      long_hold)

   
    
    
    starting_timestep +=1
    n += 1

The predicted price for 14:02 is 176.84 and the current price is 177.085


In [ ]:
#compute rmse for predictions
mse = sklearn.metrics.mean_squared_error(actual_prices, predictions)
rmse = np.sqrt(mse)
rmse